In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [1]:
#from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

#llm = ChatGroq(model="llama3-8b-8192")
llm = ChatOllama(model="llama3.2:1b")

In [3]:
from jinja2 import Template

def get_completion(prompt):
    ''' Get a completion from the Ollama API 
    Args:
        prompt (str): The prompt to send to the API
    Returns:
        str: The completion text
    '''
    #model (str): The model to use for the completion
    messages = [{"role": "user", "content": prompt}]
    response = llm.invoke(messages)
    return response

In [4]:
class PromptTemplate:
    ''' A class to represent a template for generating prompts with variables
    Attributes:
        template (str): The template string with variables
        input_variables (list): A list of the variable names in the template
    '''
    def __init__(self, template, input_variables):
        self.input_variables = input_variables
        self.template = Template(template)

    def format(self, **kwargs):
        return self.template.render(**kwargs)

In [6]:
simple_template = PromptTemplate(
    template="Provide a brief explanation of {{ topic }}.",
    input_variables=["topic"]
)

complex_template = PromptTemplate(
    template="Explain the concept of {{ concept }} in the field of {{ field }} to a {{ audience }} audience, concisely.",
    input_variables=["concept", "field", "audience"]
)

# Using the simple template
print("Simple Template Result:")
prompt = simple_template.format(topic="photosynthesis")
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")

# Using the complex template
print("Complex Template Result:")
prompt = complex_template.format(
    concept="neural networks",
    field="artificial intelligence",
    audience="beginner"
)
print(get_completion(prompt))

Simple Template Result:
content='Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose, a type of sugar. This process occurs in specialized organelles called chloroplasts, which are present in plant cells.\n\nDuring photosynthesis, light energy from the sun is absorbed by pigments such as chlorophyll, which excites an electron that is then transferred to a molecule called ATP (adenosine triphosphate). The energy from this electron transfer is used to convert carbon dioxide and water into glucose and oxygen. This process is essential for life on Earth, providing energy for plants to grow and develop, and supporting the food chain by producing oxygen as a byproduct.\n\nThe simplified equation for photosynthesis is:\n\n6 CO2 + 6 H2O + light energy → C6H12O6 (glucose) + 6 O2' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-06-26T14:33:13.4796097Z', 'done': Tru

In [7]:
#Using variables for dynamic content
# Template with conditional content
conditional_template = PromptTemplate(
    template="My name is {{ name }} and I am {{ age }} years old. "
              "{% if profession %}I work as a {{ profession }}.{% else %}I am currently not employed.{% endif %} "
              "Can you give me career advice based on this information? answer concisely.",
    input_variables=["name", "age", "profession"]
)

# Using the conditional template
print("Conditional Template Result (with profession):")
prompt = conditional_template.format(
    name="Gayaz",
    age="24",
    profession="GenAI Engineer"
)
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")

print("\nConditional Template Result (without profession):")
prompt = conditional_template.format(
    name="Gayaz",
    age="24",
    profession=""
)
print(get_completion(prompt))

Conditional Template Result (with profession):
content="As a GenAI Engineer, your role combines artificial intelligence, machine learning, and natural language processing. Here are some concise career advice points tailored to your situation:\n\n1. **Stay up-to-date with industry advancements**: Follow AI-related news, research papers, and blogs to remain current with the latest developments.\n2. **Develop skills in complementary fields**: Consider taking courses or earning certifications in computer vision, deep learning, or other related areas to enhance your expertise.\n3. **Explore entrepreneurship opportunities**: With GenAI Engineer skills, you may want to start a company or work as an AI consultant for startups, helping them navigate the field.\n4. **Network with professionals**: Attend industry events, join online communities (e.g., Kaggle, Reddit), and connect with experienced GenAI Engineers to learn from their experiences.\n5. **Pursue interdisciplinary projects**: Collabora

In [10]:
# Template for list processing
list_template = PromptTemplate(
    template="Categorize these items into groups: {{ items }}. Provide the categories and the items in each category.",
    input_variables=["items"]
)

# Using the list template
print("List Template Result:")
prompt = list_template.format(
    items="apple, banana, carrot, hammer, screwdriver, pliers, novel, textbook, magazine"
)
print(get_completion(prompt).content)

List Template Result:
Here are the categorized items:

**Fruits**

* Apple
* Banana
* Carrot

**Tools**

* Hammer
* Screwdriver
* Pliers

**Writing/Reading Materials**

* Novel
* Textbook
* Magazine


In [11]:
# Template with formatted list
list_format_template = PromptTemplate(
    template="Analyze the following list of items:\n"
              "{% for item in items.split(',') %}"
              "- {{ item.strip() }}\n"
              "{% endfor %}"
              "\nProvide a summary of the list and suggest any patterns or groupings.",
    input_variables=["items"]
)


# Using the formatted list template
print("Formatted List Template Result:")
prompt = list_format_template.format(
    items="Python, JavaScript, HTML, CSS, React, Django, Flask, Node.js"
)
print(get_completion(prompt).content)

print("\n" + "-"*50 + "\n")

Formatted List Template Result:
After analyzing the list, here's a summary and some suggestions for patterns or groupings:

**Summary:**

The list consists of 7 programming languages used in web development, with several frameworks and libraries included.

**Patterns/Grouping Suggestions:**

1. **Front-end Development Frameworks**: Python (with Django) and JavaScript (with React) are both popular front-end development frameworks.
2. **Back-end Frameworks**: The remaining three options - Flask, Node.js, and HTML/CSS - can be grouped together as back-end frameworks or technologies.
3. **Development Methodologies**: You can also group the list by development methodologies:
	* Python (with Django) suggests a "full-stack" approach, which includes front-end, back-end, database, testing, and deployment.
	* JavaScript (with React) is often used in conjunction with other frameworks like Node.js for server-side rendering or API interactions.
4. **Libraries and Tools**: You can also consider grou

In [13]:
# Template with dynamic instructions
dynamic_instruction_template = PromptTemplate(
    template="Task: {{ task }}\n"
              "Context: {{ context }}\n"
              "Constraints: {{ constraints }}\n\n"
              "Please provide a solution that addresses the task, considers the context, and adheres to the constraints.",
    input_variables=["task", "context", "constraints"]
)

# Using the dynamic instruction template
print("Dynamic Instruction Template Result:")
prompt = dynamic_instruction_template.format(
    task="Design a logo for a tech startup",
    context="The startup focuses on AI-driven healthcare solutions",
    constraints="Must use blue and green colors, and should be simple enough to be recognizable when small"
)
print(get_completion(prompt).content)

Dynamic Instruction Template Result:
Based on the context of an AI-driven healthcare startup, I've designed a logo concept that incorporates blue and green colors while being simple and recognizable even at a small size.

**Logo Concept:**

The logo features a stylized combination of two intertwined elements: a brain (symbolizing intelligence and health) and a wave (representing movement and fluidity).

**Color Scheme:**

* Primary color: #34A85A (a bright, calming blue)
* Secondary color: #8BC34A (a fresh, energetic green)

**Logo Design:**

The logo consists of two connected shapes:

1. A stylized brain with a subtle wave pattern along its surface, incorporating the secondary color #8BC34A.
2. The top half of the brain is outlined in primary color #34A85A.

The wave element is simple and smooth, allowing it to flow around the brain without disrupting the design. This creates a sense of movement and fluidity, emphasizing the AI-driven healthcare aspect.

**Icon Variations:**

To ensur